# EuroSAT Classification — Train classifier

This notebook fine-tunes a **ResNet-50** model (or trains a SimpleCNN baseline) on the EuroSAT dataset.

It follows the experimental setup described in:

'*EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification*' by Helber et al., 
IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 2019.

### 1. Setup environment and imports

In [ ]:
import sys, os
if not hasattr(sys, "frozen"):
    os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"

import torch
from torchvision import models
import torch.nn as nn

# Add project root to path
sys.path.append(os.path.abspath("..")) 
from src.training.train import train_model
from src.training.evaluate import evaluate_model, plot_confusion_matrix

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Using device: {device}")

- Configuration

In [ ]:
model_name = "resnet50"   # Options: "simplecnn", "resnet18", "resnet50"
data_dir = "../data/raw"  
checkpoint_dir = "../experiments/checkpoints"
epochs = 100
early_stopping = 10
batch_size = 256
lr = 1e-4

checkpoint_path = os.path.join(checkpoint_dir, f"{model_name}_adv.pth")


### 2. Train baseline SimpleCNN / ResNet18 / ResNet50 model

In [ ]:
print(f"\n🛰️ Training {model_name.upper()} on EuroSAT dataset...")

model, classes = train_model(
    data_dir=data_dir,
    batch_size=batch_size,
    epochs=epochs,
    lr=lr,
    model_name=model_name,
    output_dir=checkpoint_dir,
    early_stopping_patience=early_stopping,
)

print(f"\n✅ Training completed. Number of classes: {len(classes)}")
print(f"Model checkpoint saved to: {checkpoint_path}")

### 3. Evaluate final metrics on Test Dataset

In [ ]:
print("\n📊 Evaluating model on test set...")

metrics = evaluate_model(
    model_path=checkpoint_path,
    data_dir=data_dir,
    batch_size=batch_size,
    model_name=model_name,
    device=device
)

- Metrics

In [ ]:
print("\n📈 Test Set Performance:")
print(f"Accuracy:  {metrics['accuracy']*100:.2f}%")
print(f"Loss:      {metrics['loss']:.4f}")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall:    {metrics['recall']:.4f}")
print(f"F1-score:  {metrics['f1']:.4f}")

print("\n🔍 Classification metrics per class:\n")
print(metrics["classification_report"])


- Confusion Matrix

In [ ]:
print("\n🧭 Confusion Matrix:")
plot_confusion_matrix(metrics["confusion_matrix"], metrics["class_names"])